In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

simulator = AerSimulator()

def pad(n1, n2) :
    if len(n1) < len(n2) :
        n1 = "0"*(abs(len(n2) - len(n1))) + n1
    elif len(n2) < len (n1) :
        n2 = "0"*(abs(len(n2) - len(n1))) + n2
    return n1, n2

def inpt() :
    while True :
        try :
            n1, n2 = [bin(int(input(f"Give number {i}: ")))[2:] for i in range(1,3)]
            pad(n1, n2)
            break
        except :
            print("Invalid input. Please try again!")
    return n1, n2


def complement(n) :
    n1, n2 = pad(n, '1')
    circuit = QuantumCircuit(len(n1))
    for i, bit in enumerate(n1) :
        if bit == '0':
            circuit.x(i)
    circuit.measure_all()
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=1)
    result = job.result()
    counts = result.get_counts(compiled_circuit)
    for key in counts:
        n1 = key[::-1]    
    return add(n1, n2)[0]


def bit_adder(a, b, cin=0) :
    circuit = QuantumCircuit(5, 2)
    if a == 1: circuit.x(0)
    if b == 1: circuit.x(1)
    if cin == 1: circuit.x(2)
    circuit.cx(0, 4)
    circuit.cx(1, 4)
    circuit.cx(2, 4)
    circuit.ccx(0, 1, 3)
    circuit.ccx(1, 2, 3)
    circuit.ccx(0, 2, 3)
    circuit.measure([3, 4], [0, 1])
    compiled_circuit = transpile(circuit, simulator)
    job = simulator.run(compiled_circuit, shots=1)
    result = job.result()
    counts = result.get_counts(compiled_circuit)
    for key in counts:
        output = key
    return output


def add(n1, n2) :
    n1, n2 = pad(n1, n2)
    n1, n2 = list(n1)[::-1], list(n2)[::-1]
    
    cin = 0
    result = ''

    for d in zip(n1, n2):
        dsum, cout = bit_adder(int(d[0]), int(d[1]), cin)
        cin = int(cout)
        result = dsum + result

    if cout == '1' : result = cout + result
    return result, cout


def subtract(n1, n2) :
    n1, n2 = pad(n1, n2)
    result, cout = add(n1, complement(n2))
    if cout == '1' : result = result[1:]
    elif cout == '0' : result = '-' + complement(result)
    return result


while True :
    opr = input('''What would you like to do?\n
    1) + for addition\n
    2) - for subtraction\n
    Choose (+ or -): ''')

    match opr:
        case '+':
            n1, n2 = inpt()
            result = add(n1, n2)[0]
            print(f"{int(n1, 2)} + {int(n2, 2)} = {int(result, 2)}")
            break
        case '-':
            n1, n2 = inpt()
            result = subtract(n1, n2)
            print(f"{int(n1, 2)} - {int(n2, 2)} = {int(result, 2)}")
            break
        case _:
            print("Invalid input. Please choose again!")

3 + 7 = 10
